In [2]:
# fuzzywuzzy를 사용하기 위해 필요한 패키지를 설치하기
!pip3 install fuzzywuzzy

# fuzzywuzzy를 사용하기 위한 라이브러리 불러오기
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# inpurt_str: 기준이 되는 문자
input_str = "US"

# choices: 비교하고자 하는 문자 리스트
choices = ["US!", "U.S.", "USA", "United States", "SU"]

#input_str 문자를 choices 안에 있는 각 문자와 비교한 뒤 상위 3개 선택
process.extract(input_str, choices, limit=3)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[('US!', 100), ('USA', 90), ('U.S.', 80)]

In [11]:
choices = ["Deluxe Room, 2 Queen Bed",
           "Deluxe Room, City View",
           "Club Room, 1 King Bed",
           "Junior Suite, 1 King Bed",
           "King Room, Deluxe"]

process.extract("Deluxe 1 King Room", choices)

[('King Room, Deluxe', 95),
 ('Club Room, 1 King Bed', 75),
 ('Deluxe Room, 2 Queen Bed', 72),
 ('Deluxe Room, City View', 72),
 ('Junior Suite, 1 King Bed', 48)]

In [12]:
# 라이브러리 불러오기
import pandas as pd
import numpy as np

# choices를 데이터 타입으로 변환
df = pd.DataFrame({'룸타입': choices})
df

,룸타입
0,"Deluxe Room, 2 Queen Bed"
1,"Deluxe Room, City View"
2,"Club Room, 1 King Bed"
3,"Junior Suite, 1 King Bed"
4,"King Room, Deluxe"


In [13]:
input_str = "Deluxe Room, 2 Queen Bed"
process.extract(input_str, df['룸타입'])


[('Deluxe Room, 2 Queen Bed', 100, 0),
 ('King Room, Deluxe', 77, 4),
 ('Club Room, 1 King Bed', 67, 2),
 ('Deluxe Room, City View', 66, 1),
 ('Junior Suite, 1 King Bed', 42, 3)]

In [14]:
# input_str을 제외했을 때, 가장 비 슷한 단어 선택(top_score)
top_score = process.extract(input_str, df['룸타입'])[1]
print(top_score)

('King Room, Deluxe', 77, 4)


In [15]:
threshold = 70

# 이 비슷한 단어의 스코어가 임계값보다 높으면 단어를 선택하고 그렇지 않은 경우 결측치로 표현
top_score[0] if top_score[1] > threshold else np.nan

'King Room, Deluxe'

In [16]:
# 위 과정을 함수로 표현
def get_match(input_str, threshold):
  """
  input_str을 df['룸타입']의 후보와 비교한 후
  가장 비슷한 값이 threshold보다 높으면 그 값을 반환하고,
  그렇지 않으면 결측치로 반환
  """
  top_score = process.extract(input_str, df['룸타입'])[1]
  top_score = top_score[0] if top_score[1] > threshold else np.nan
  return top_score

df['룸타입_matched'] = [get_match(row, 70) for row in df['룸타입']]

df

,룸타입,룸타입_matched
0,"Deluxe Room, 2 Queen Bed","King Room, Deluxe"
1,"Deluxe Room, City View","King Room, Deluxe"
2,"Club Room, 1 King Bed",NaN
3,"Junior Suite, 1 King Bed",NaN
4,"King Room, Deluxe","Deluxe Room, 2 Queen Bed"
